<a href="https://colab.research.google.com/github/asad-777/Piaic_Gen-Ai/blob/main/University_Faq_Bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain google_genai langchain_google_genai chromadb langchain-chroma -q -U

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 72.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 4.47.1 requires tokenizers<0.22,>=0.21, but you have tokenizers 0.20.3 which is incompatible.


In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI,GoogleGenerativeAIEmbeddings

In [ ]:
from google.colab import userdata
api_key=userdata.get('GEMINI_API_KEY')
pinecone_api=userdata.get('PINECONE_API')

In [ ]:
file_path = "/content/data.csv"

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv(file_path)

In [ ]:
data = df.apply(lambda row: f"question: {row['Question']}\nAnswer: {row['Answer']}", axis=1).tolist()

In [ ]:
full_text = "\n".join(data)

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=50)

In [ ]:
chunks = text_splitter.create_documents([full_text])

In [ ]:
print(len(chunks))

267


In [ ]:
import os
os.environ["GOOGLE_API_KEY"] = api_key

In [ ]:
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [ ]:
print(len(chunks))

267


In [ ]:
import chromadb
import time

client = chromadb.Client()
collection = client.create_collection("university_data_2")

batch_size = 50

for i in range(0, len(chunks), batch_size):
    batch = chunks[i:i + batch_size]
    chunk_embeddings = embeddings.embed_documents([chunk.page_content for chunk in batch])

    collection.add(
        documents=[chunk.page_content for chunk in batch],
        embeddings=chunk_embeddings,
        ids=[str(i + idx) for idx in range(len(batch))],
    )

    time.sleep(10)


In [ ]:
import chromadb
from langchain_chroma import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings

# Initialize the client and load the collection
client = chromadb.Client()
collection = client.get_collection("university_faq_data")

# Initialize the embedding function
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

# Create a Chroma vector store using the correct directory
vector_store = Chroma(persist_directory="path_to_persist_directory", embedding_function=embeddings.embed_documents)

# Perform similarity search
query = "fees"
retrieved_docs = vector_store.similarity_search(query, k=5)

# Print the retrieved documents
for doc in retrieved_docs:
    print(doc.page_content)

AttributeError: 'function' object has no attribute 'embed_query'